신경망 학습: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득
지표: 신경망이 학습할 수 있도록 도와주는 것

# 데이터에서 학습

신경망은 매개변수의 값을 데이터를 기반으로 자동으로 결정

## 데이터 주도 학습

- 기계학습
    - 사람의 개입 최소화, 수집 데이터로부터 패턴 도출
    - ex) 특정 이미지로부터 숫자의 판별 => 알고리즘으로 해결하는 데에 어려움 존재
    - 밑바닥부터 설계하는 것이 아닌, 이미지로부터 특징을 추출하여 패턴을 기계에 학습시킴 (다만, 이미지 변환시의 특징은 사람이 설계함)
    - 이미지 특징: 벡터 (CV: SIFT, SURF, HOG) => 변환된 벡터로 SVM, KNN 등을 이용하여 학습 진행
- 신경망(딥러닝)
    - 이미지를 있는 그대로 학습
    - 이미지의 중요한 특징도 기계가 스스로 학습

## 훈련 데이터와 시험 데이터
훈련 데이터만 사용하여 학습, 시험 데이터로 훈련한 모델의 성능 평가
나누는 이유: 범용 능력 (아직 확인하지 못한 데이터로도 문제를 풀어내는 능력)을 제대로 평가하기 위해
 = 과적합 (overfitting)을 방지하기 위해

# 손실 함수
신경망 학습: 현재 상태를 하나의 지표로 표현
하나의 지표를 기준으로 최적 매개변수 값을 탐색
신경망에서의 지표 = 손실 함수: (주로 오차제곱합, 교차 엔트로피 오차 등)

## 오차제곱합

$$E = \frac{1}{2}\sum\limits_{k}(y_k - t_k)^2$$

y_k: 신경망의 출력
t_k: 정답 레이블
k: 데이터의 차원 수

In [1]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [6]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

In [7]:
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [8]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

## 교차 엔트로피 오차 (CEE)

$$E = -\sum\limits_{k}t_k\log y_k$$

log: 밑이 e인 자연로그
y_k: 신경망의 출력
t_k: 정답 레이블 (정답의 인덱스만 1, 나머지는 0)

정답에 해당하는 출력이 커질수록 0에 다가감, 출력이 1일때 0이 됨

In [10]:
def cross_entropy_error(y, t):
    delta = 1e-7 # np.log(0)은 -inf가 되기 때문에, 아주 작은 값을 더해 0이 되지 않도록 조정
    return -np.sum(t * np.log(y + delta))

In [11]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [12]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

## 미니배치 학습

훈련 데이터 모두에 대한 손실 함수 값들의 합을 지표로 함
$$E = -\frac{1}{N}\sum\limits_{n}\sum\limits_{k}t_{nk}\log y_{nk}$$

교차 엔트로피 오차를 N개의 데이터로 확장하고 N으로 나누어 정규화 (평균 손실 함수)
미니배치
- N이 커지면 현실에 적용하기 어려움
- 무작위로 n장을 뽑아서 학습하는 것

In [13]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from datasets.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

print(X_train.shape, y_train.shape)

(60000, 784) (60000, 10)


무작위로 10장만 빼내기

In [14]:
train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = X_train[batch_mask]
y_batch = y_train[batch_mask]

## 교차 엔트로피 오차 구현

In [15]:
def cross_entropy_error(y, t): # y: 신경망의 출력, t: 정답 레이블
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [16]:
def cross_entropy_error(y, t): # y: 신경망의 출력, t: 숫자
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

## 손실 함수 설정 이유
정확도를 두고 손실 함수를 사용하는 이유?
- 최적의 매개변수 탐색 시 손실 함수의 값을 가능한 한 작게 하는 매개변수 값 탐색 (미분 계산, 서서히 갱신)
- 정확도를 지표로 삼게 되면 미분 값이 대부분의 장소에서 0이 되어 매개변수 갱신이 불가능해지기 때문
    - 정확도는 계단 함수처럼 불연속적인 값을 가짐 (전체 데이터 중 특정 데이터의 일치 정도)
    - 손실 함수는 매개변수가 변화하면 변화에 대응하는 값이 연속적으로 변화
- 계단 함수(ex 정확도)를 활성화 함수로 사용하면 대부분의 장소에서 미분이 0이기 때문에 학습이 이루어지지 않음
- 시그모이드 함수는 어느 장소라도 미분하여 0이 되지 않음 => 기울기가 0이 되지 않기 때문에 올바른 학습 가능